<a href="https://colab.research.google.com/github/Aswini-J/Connect4/blob/main/connect4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-sgiwngpb
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-sgiwngpb
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 801584cceb559adc54e828ebe9b385c5f53fe70f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10743 sha256=3a570d3ab74e9051a615aa78d54722c78a69a0c83a1d407fccde14c37a30e96c
  Stored in directory: /tmp/pip-ephem-wheel-cache-_y8eq6x4/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpao9mkehz".


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

#define ROWS 6
#define COLS 7
#define WINNING_COUNT 4

// Kernel to make a move
__global__ void makeMove(int *board, int col, int player) {
    int row = 0;
    while (row < ROWS && board[row * COLS + col] == 0) {
        row++;
    }
    if (row > 0) {
        board[(row - 1) * COLS + col] = player;
    }
}

// Kernel to check for a win
__global__ void checkWin(int *board, bool *winFound) {
    *winFound = false;

    // Check horizontal
    for (int r = 0; r < ROWS; r++) {
        for (int c = 0; c <= COLS - WINNING_COUNT; c++) {
            int count = 1;
            for (int k = 1; k < WINNING_COUNT; k++) {
                if (board[r * COLS + c + k] == board[r * COLS + c]) {
                    count++;
                } else {
                    break;
                }
            }
            if (count == WINNING_COUNT) {
                *winFound = true;
                return;
            }
        }
    }

    // Check vertical
    for (int c = 0; c < COLS; c++) {
        for (int r = 0; r <= ROWS - WINNING_COUNT; r++) {
            int count = 1;
            for (int k = 1; k < WINNING_COUNT; k++) {
                if (board[(r + k) * COLS + c] == board[r * COLS + c]) {
                    count++;
                } else {
                    break;
                }
            }
            if (count == WINNING_COUNT) {
                *winFound = true;
                return;
            }
        }
    }

    // Check diagonals (bottom-left to top-right)
    for (int r = 0; r <= ROWS - WINNING_COUNT; r++) {
        for (int c = 0; c <= COLS - WINNING_COUNT; c++) {
            int count = 1;
            for (int k = 1; k < WINNING_COUNT; k++) {
                if (board[(r + k) * COLS + c + k] == board[r * COLS + c]) {
                    count++;
                } else {
                    break;
                }
            }
            if (count == WINNING_COUNT) {
                *winFound = true;
                return;
            }
        }
    }

    // Check diagonals (top-left to bottom-right)
    for (int r = WINNING_COUNT - 1; r < ROWS; r++) {
        for (int c = 0; c <= COLS - WINNING_COUNT; c++) {
            int count = 1;
            for (int k = 1; k < WINNING_COUNT; k++) {
                if (board[(r - k) * COLS + c + k] == board[r * COLS + c]) {
                    count++;
                } else {
                    break;
                }
            }
            if (count == WINNING_COUNT) {
                *winFound = true;
                return;
            }
        }
    }
}

void printBoard(int *board) {
    for (int r = 0; r < ROWS; r++) {
        for (int c = 0; c < COLS; c++) {
            printf("%d ", board[r * COLS + c]);
        }
        printf("\n");
    }
    printf("\n");
}

int main() {
    int *d_board;
    bool *d_winFound;
    int board[ROWS * COLS] = {0};
    bool winFound = false;

    cudaMalloc(&d_board, sizeof(int) * ROWS * COLS);
    cudaMalloc(&d_winFound, sizeof(bool));

    cudaMemcpy(d_board, board, sizeof(int) * ROWS * COLS, cudaMemcpyHostToDevice);
    cudaMemcpy(d_winFound, &winFound, sizeof(bool), cudaMemcpyHostToDevice);

    // Example moves
    makeMove<<<1, 1>>>(d_board, 3, 1); // Player 1 moves
    cudaDeviceSynchronize();
    makeMove<<<1, 1>>>(d_board, 4, 2); // Player 2 moves
    cudaDeviceSynchronize();
    makeMove<<<1, 1>>>(d_board, 3, 1); // Player 1 moves
    cudaDeviceSynchronize();
    makeMove<<<1, 1>>>(d_board, 4, 2); // Player 2 moves
    cudaDeviceSynchronize();
    makeMove<<<1, 1>>>(d_board, 3, 1); // Player 1 moves
    cudaDeviceSynchronize();
    makeMove<<<1, 1>>>(d_board, 4, 2); // Player 2 moves
    cudaDeviceSynchronize();
    makeMove<<<1, 1>>>(d_board, 3, 1); // Player 1 moves
    cudaDeviceSynchronize();

    // Check for win
    checkWin<<<1, 1>>>(d_board, d_winFound);
    cudaDeviceSynchronize();

    cudaMemcpy(&winFound, d_winFound, sizeof(bool), cudaMemcpyDeviceToHost);

    if (winFound) {
        printf("Player has won!\n");
    } else {
        printf("No win yet.\n");
    }

    // Print the board
    cudaMemcpy(board, d_board, sizeof(int) * ROWS * COLS, cudaMemcpyDeviceToHost);
    printBoard(board);

    cudaFree(d_board);
    cudaFree(d_winFound);
    return 0;
}


Player has won!
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 0 1 0 0 0 
0 0 0 1 2 0 0 
0 0 0 1 2 0 0 
0 0 0 1 2 0 0 


